# Trump Golf Statistics

This site keeps Trump's Golf statistics.

In [1]:
import IPython
url="http://trumpgolfcount.com/#page-top"
IPython.display.HTML('<iframe src ='+url+' width=700 height=350></iframe>')

They thoughtfully provide a json stream but it is not self-descriptive.  Thankfully they have a comma separated value (csv) link the includes headers.

## Get and Wrangle the Date

First we pull down the data and do a bit of wrangling on it.

In [2]:
import requests
csvUrl="http://trumpgolfcount.com/downloadoutings"
golfCsv=requests.get(csvUrl).text
import pandas as pd
from io import StringIO
with StringIO(golfCsv) as sioCsv:
    gDf=pd.DataFrame.from_csv(sioCsv)
gDf.sort_index(inplace=True)
gDf['Arrival Time']=pd.to_datetime(gDf['Arrival Time'])
gDf['Depart Time']=pd.to_datetime(gDf['Depart Time'])
gDf['ndDays']=gDf.reset_index()['Golf Date'].diff().fillna(pd.Timedelta(1)).values
gDf['ndDays']=gDf['ndDays'].apply(lambda x: x.days)

## Plot the Data

I am going to play a bit with ```bokeh``` here. Yet another graphics library ot learn.  However, this one is web native so maybe it will be worth it.

In [4]:
from bokeh.io import show

In [5]:
from bokeh.io import output_notebook
from bokeh.plotting import figure
from bokeh.models import Span,Label
TsDay=24*3600*1000.
p=figure(title="Preceeding Days with no Trump Golf",
       plot_width=550,plot_height=300,
         x_axis_type="datetime",)
p.vbar(source=gDf.reset_index(),
       x='Golf Date',top='ndDays',width=1,
      )
Today=pd.tslib.Timestamp.today().timestamp()*1000
vline=Span(location=Today,dimension='height',
           line_width=2,line_color='red',line_dash='dashed'
          )
p.renderers.extend([vline])
label=Label(x=Today-10*TsDay,y=10,
           text="Today",render_mode='css',
            angle=270,angle_units='deg',
           background_fill_color='white')
p.add_layout(label)
output_notebook();show(p)

Loading BokehJS ...